In [1]:
import math
import pandas as pd 
import numpy as np 
from collections import namedtuple
from dataclasses import dataclass
from numba import jit,njit

from __future__ import print_function
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

@dataclass(frozen=True,eq=True)
class Customer:
    index:int
    demand:int
    x:float
    y:float
        
@njit
def length_xy(x1,y1,x2,y2):
    return math.sqrt((x1-x2)*(x1-x2) + (y1-y2)*(y1-y2))

def length(point1, point2):
    return length_xy(point1.x,point1.y,point2.x,point2.y)

In [2]:
def read_data(input_data):
    lines = input_data.split('\n')

    parts = lines[0].split()
    customer_count = int(parts[0])
    vehicle_count = int(parts[1])
    vehicle_capacity = int(parts[2])
    
    customers = []
    for i in range(1, customer_count+1):
        line = lines[i]
        parts = line.split()
        customers.append(Customer(i-1, int(parts[0]), float(parts[1]), float(parts[2])))
    depot = customers[0]
    return customers,customer_count,vehicle_count,vehicle_capacity,depot
file_name = "./data/vrp_16_3_1"
input_data = open(file_name).read()
customers,customer_count,vehicle_count,vehicle_capacity,depot = read_data(input_data)

In [3]:
customer_demands = list(map(lambda x:x.demand,customers))
vehicle_capacities = [vehicle_capacity] * vehicle_count

In [4]:
distances_matrix = np.zeros((customer_count,customer_count))
for i in range(customer_count):
    for j in range(customer_count):
        distances_matrix[i,j] = length(customers[i],customers[j]) * 100
# distances_matrix = distances_matrix.astype("int32")
distances_matrix = np.around(distances_matrix)
# distances_matrix = np.ceil(distances_matrix)

In [5]:
pywrapcp.RoutingModel.__doc__

In [6]:
@njit
def distance_callback(f,t): return distances_matrix[f,t]
def demand_callback(f,t): return customer_demands[f]

In [7]:
routing = pywrapcp.RoutingModel(customer_count,vehicle_count,depot.index)
routing.SetArcCostEvaluatorOfAllVehicles(distance_callback)
routing.AddDimensionWithVehicleCapacity(demand_callback,0,vehicle_capacities,True,"Capacity")

True

In [8]:
search_parameters = pywrapcp.RoutingModel.DefaultSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.lns_time_limit_ms = 100
search_parameters.optimization_step = 1
assignment = routing.SolveWithParameters(search_parameters)
# print(assignment)

In [11]:
search_parameters

first_solution_strategy: PATH_CHEAPEST_ARC
use_filtered_first_solution_strategy: true
local_search_operators {
  use_relocate: true
  use_relocate_pair: true
  use_exchange: true
  use_cross: true
  use_two_opt: true
  use_or_opt: true
  use_lin_kernighan: true
  use_make_active: true
  use_make_inactive: true
  use_swap_active: true
  use_node_pair_swap_active: true
}
guided_local_search_lambda_coefficient: 0.1
optimization_step: 1
solution_limit: 9223372036854775807
time_limit_ms: 9223372036854775807
lns_time_limit_ms: 100
use_light_propagation: true
fingerprint_arc_cost_evaluators: true

In [9]:
def print_solution(routing, assignment):
    """Print routes on console."""
    total_dist = 0
    result = ""
    for vehicle_id in range(vehicle_count):
        index = routing.Start(vehicle_id)
        plan_output = ""
        route_dist = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = routing.IndexToNode(index)
            next_node_index = routing.IndexToNode(assignment.Value(routing.NextVar(index)))
            route_dist += routing.GetArcCostForVehicle(node_index, next_node_index, vehicle_id)
            route_load += customer_demands[node_index]
            plan_output += '{0} ' .format(node_index)
            index = assignment.Value(routing.NextVar(index))

        node_index = routing.IndexToNode(index)
        total_dist += route_dist
        plan_output += ' {0}\n'.format(node_index, route_load)
        
        result += plan_output
        plan_output =  f"{vehicle_id} : " + plan_output
        plan_output += 'Distance of the route: {0}m\n'.format(route_dist)
        plan_output += 'Load of the route: {0}\n'.format(route_load)
        print(plan_output)
    print('Total Distance of all routes: {0}m'.format(total_dist))
    return f"{total_dist} 0\n" + result 
result = print_solution(routing,assignment)

0 : 0 2 3 8 7  0
Distance of the route: 9788m
Load of the route: 88

1 : 0 6 14 13 4 15 10  0
Distance of the route: 11811m
Load of the route: 83

2 : 0 1 11 9 5 12  0
Distance of the route: 6843m
Load of the route: 87

Total Distance of all routes: 28442m


In [47]:
result_file = open(f"{file_name}.result","w")
result_file.write(result)
result_file.close()